In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from utils import MultipleRegressionWithSoftmax, EmbeddingsDataset2

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr

In [2]:
n_classes = 2
modality = 'music' # 'music', 'speech', or 'video'
which = 'openl3' # 'mfcc', 'msd' or 'openl3' for music, 'slow_fast' for video, 'hubert' for speech
voice = True 

fn_suffix = {
    'music': {
        'mfcc': '',
        'msd': '_backend', 
        'openl3': '_music', # '_music' or '_env'
    },
    'video': {
        'slow_fast': '_slow', # '_slow' or '_fast'
    },
    'speech': {
        'hubert': '_wave_encoder', # '_wave_encoder' or '_transformer'
    }
}

embedding_dimensions = {
    'video': {
        'slow_fast': 2048 if fn_suffix['video']['slow_fast']=='_slow' else 256,
    },
    'music': {
        'mfcc': 60,
        'msd': 256,
        'openl3': 512,
    },
    'speech': {
        'hubert': 1024 if fn_suffix['speech']['hubert']=='_transformer' else 512,
    }
}

## Load ground truth

In [3]:
groundtruth_df = pd.read_csv("groundtruth_merged.csv")
groundtruth_df.set_index("stimulus_id", inplace=True)
groundtruth_df.head()

,product_category,filming_location,target,interaction,voice_type,voice_age,voice_gender,voice_exagg,asian,black,...,description,upload_date,duration,view_count,categories,tags,like_count,requested_subtitles,download,error_logs
stimulus_id,,,,,,,,,,,,,,,,,,,,,
ndzo2ZIWfiQ,High-tech Interactive Playmates and Robotics,Non-specific,Girls/women,They do not interact with each other or there ...,BOTH spoken and sung,Adults (including young adults),Feminine,"No, all voices are normal-sounding",No,No,...,NaN,20120907,20,3047,['Autos & Vehicles'],['122975'],5,"{'en': {'ext': 'vtt', 'url': 'https://www.yout...",True,NaN
yRUiwRKk6QM,High-tech Interactive Playmates and Robotics,Indoors,Mixed,They do not interact with each other or there ...,Spoken,Adults (including young adults),Masculine,Yes a masculine voice is gender exaggerated,No,Yes,...,Now you can train like a Jedi! Use the power o...,20170920,34,25861,['Entertainment'],"['Smyths Toys', 'Toys', '(Industry)kids', 'Sta...",46,NaN,True,NaN
3ysC1-foJT4,"Apparel, Fashion, Accessories, Cosmetics, Cost...",Indoors,Girls/women,They are working or playing together in a coop...,Sung,Adults (including young adults),Feminine,Yes a feminine voice is gender exaggerated,No,No,...,Just add water to make endless crystal creatio...,20180802,30,2545,['Entertainment'],"['Smyths Toys', 'Toys (Industry)', 'kids', 'sm...",19,NaN,True,NaN
cYszuGaptkk,"Action Figures, Battling Toys and Toy Weapons",Non-specific,Mixed,They are working or playing together in a coop...,Spoken,Adults (including young adults),Feminine,"No, all voices are normal-sounding",No,Yes,...,Create your own exciting dino rescue missions ...,20201028,21,2035695,['Entertainment'],"['Smyths Toys', 'Toys (Industry)', 'kids', 'sm...",51,"{'en': {'ext': 'vtt', 'url': 'https://www.yout...",True,NaN
2LZjLBipdfI,Dolls,Indoors,Girls/women,They do not interact with each other or there ...,BOTH spoken and sung,Adults (including young adults),Feminine,Yes a feminine voice is gender exaggerated,No,No,...,Fabio & Fabia's Hair salon is the most loved p...,20201106,15,1834,['Entertainment'],"['Smyths Toys', 'Toys (Industry)', 'kids', 'sm...",10,NaN,True,NaN


In [4]:
mid_level_features = pd.read_csv("mid_level_features.csv").drop(columns=["target"])
mid_level_features.set_index("stimulus_id", inplace=True)
mid_level_features.head()

,Electric/Acoustic,Distorted/Clear,Many Instruments/Few Instruments,Loud/Soft,Heavy/Light,High pitch/Low pitch,Wide pitch variation/Narrow pitch variation,Punchy/Smooth,Harmonious/Disharmonious,Clear melody/No melody,Repetitive/Non-repetitive,Complex rhythm/Simple rhythm,Fast tempo/Slow tempo,Dense/Sparse,Strong beat/Weak beat
stimulus_id,,,,,,,,,,,,,,,
-BaTPbE0Gdo,3.666667,6.500000,4.333333,5.833333,6.666667,2.666667,4.666667,6.000000,1.500000,2.166667,4.333333,5.666667,3.500000,4.166667,4.166667
-KKsNKY4V8k,3.000000,5.166667,2.500000,2.500000,5.000000,2.500000,3.666667,4.000000,2.666667,2.500000,2.666667,5.000000,2.500000,3.166667,3.166667
-Mqc2csT3ZM,2.666667,3.500000,4.666667,4.666667,5.333333,2.666667,4.666667,3.500000,3.500000,3.333333,2.833333,5.333333,3.333333,3.666667,4.000000
-NEHGAMiA2I,4.833333,6.166667,5.500000,5.666667,6.166667,3.666667,4.500000,5.500000,2.333333,3.666667,2.833333,4.833333,4.333333,4.500000,4.333333
-SEKfzdaIK0,3.666667,3.166667,5.333333,3.500000,2.833333,4.166667,4.500000,2.166667,3.000000,3.000000,2.666667,4.833333,2.666667,4.000000,3.000000


In [8]:
# perform min-max scaling [0, 1] of all mid-level features
mid_level_features = (mid_level_features - mid_level_features.min()) / (mid_level_features.max() - mid_level_features.min())

In [10]:
not_found = 0
for stimulus_id in groundtruth_df.index:
    if not os.path.exists(f"{modality}/embeddings_{which}/{stimulus_id}{fn_suffix[modality][which]}.npy"):
        print(f"Embedding for {stimulus_id} not found")
        not_found += 1

assert not_found == 0

## Load embeddings

In [11]:
embedding_dim = embedding_dimensions[modality][which]

X = np.empty((groundtruth_df.shape[0], embedding_dim))
y_reg = np.empty((mid_level_features.shape[0], mid_level_features.shape[1]))

for i,stimulus_id in enumerate(groundtruth_df.index):
    embedding = np.load(f"{modality}/embeddings_{which}{'' if voice else '_novoice'}/" +
                        f"{stimulus_id}{fn_suffix[modality][which]}.npy")
    X[i] = embedding.mean(axis=0)
    y_reg[i] = mid_level_features.loc[stimulus_id].values

X.shape, y_reg.shape

((606, 512), (606, 15))

In [12]:
classes = ["Girls/women", "Boys/men"] if n_classes==2 else ["Girls/women", "Mixed", "Boys/men"]
mask = groundtruth_df.target.isin(classes) 

X = X[mask]
y_cls = groundtruth_df.target[mask].values
y_reg = y_reg[mask]

# convert to integers
le = LabelEncoder()
y_cls = le.fit_transform(y_cls)
y_cls.shape

(312,)

In [13]:
params = {
    "input_dim": X.shape[1], 
    "n_regressions": y_reg.shape[1], 
    "output_dim": n_classes
    }

kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []
f1s = []
r2s = []
pearsons = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_reg_train, y_reg_test = y_reg[train_index], y_reg[test_index]
    y_cls_train, y_cls_test = y_cls[train_index], y_cls[test_index]
    
    train_dataset = EmbeddingsDataset2(X_train, y_reg_train, y_cls_train)
    test_dataset = EmbeddingsDataset2(X_test, y_reg_test, y_cls_test)
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=10)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=10)

    model = MultipleRegressionWithSoftmax(**params)
    
    checkpoint_callback = ModelCheckpoint(monitor='val_loss')
    trainer = Trainer(max_epochs=100,
                      callbacks=[checkpoint_callback, EarlyStopping(monitor='val_loss', patience=20)],
                      enable_progress_bar = False,
                      accelerator='gpu',
                      devices=1)
    trainer.fit(model, train_loader, test_loader)

    # load best model
    model = model.load_from_checkpoint(checkpoint_callback.best_model_path, **params)
    
    model.eval()
    with torch.no_grad():
        y_reg_pred, out_cls = model(torch.from_numpy(X_test).float())
    
    y_cls_pred = torch.argmax(out_cls, dim=1).numpy()
    
    accuracies.append(accuracy_score(y_cls_test, y_cls_pred))
    f1s.append(f1_score(y_cls_test, y_cls_pred, average='weighted'))

    r2_values = r2_score(y_reg_test, y_reg_pred, multioutput='raw_values')
    r2s.append(r2_values)

    r = [pearsonr(y_reg_test[:,i], y_reg_pred[:,i])[0] for i in range(y_reg_test.shape[1])]
    pearsons.append(r)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]

  | Name    | Type   | Params
-----------------------------------
0 | linear  | Linear | 65.7 K
1 | linear2 | Linear | 1.9 K 
2 | linear3 | Linear | 258   
-----------------------------------
67.9 K    Trainable params
0         Non-trainable params
67.9 K    Total params
0.271     Total estimated model params size (MB)
/homes/lm004/.conda/envs/embeddings_pipeline_dev/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (32) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(
Exception ignored in: <function _releaseLock at 0x7fbfc0001e10>
Traceback (most recent call last):
  File "/homes/lm004/.conda/envs/e

In [ ]:
print(f"Accuracy: {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}")
print(f"F1: {np.mean(f1s):.2f} ± {np.std(f1s):.2f}")

In [ ]:
# transpose to get r2s per feature
r2s = np.array(r2s).T

for i, r2 in enumerate(r2s):
    print(f"R2 for {mid_level_features.columns[i]}: {np.mean(r2):.2f} ± {np.std(r2):.2f}")

In [ ]:
# transpose to get pearsons per feature
pearsons = np.array(pearsons).T

for i, r in enumerate(pearsons):
    print(f"Pearson's r for {mid_level_features.columns[i]}: {np.mean(r):.2f} ± {np.std(r):.2f}")